In [3]:
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
from glob import glob

class GaussianNaiveBayes:
    def __init__(self, epsilon=1e-3):
        self.classes = None
        self.mean = None
        self.var = None
        self.priors = None
        self.epsilon = epsilon

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]
        self.mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self.var = np.zeros((n_classes, n_features), dtype=np.float64)
        self.priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0) + self.epsilon
            self.priors[idx] = X_c.shape[0] / float(X.shape[0])

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []
        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            class_conditional = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


def extract_hog_features(image_path):
    try:
        image = imread(image_path, as_gray=True)
        image_resized = resize(image, (128, 64))
        hog_features = hog(image_resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        return hog_features
    except Exception as e:
        print(f"No se pudo procesar la imagen {image_path}: {e}")
        return None

features = []
labels = []

# Diccionario para asociar carpetas con etiquetas
label_dict = {
    'glass_tiles': 0,
    'hairs': 1,
    'flowers': 2,
    'oil_painting': 3,
    'jelly_beans': 4,
    'paints': 5,
    'photo_and_face': 6,
    'pompoms': 7,
    'sponges': 8,
    'stuffed_toys': 9
}


Accuracy (Manual): 1.0
Confusion Matrix (Manual):
[[8 0 0 0 0 0 0 0 0 0]
 [0 8 0 0 0 0 0 0 0 0]
 [0 0 8 0 0 0 0 0 0 0]
 [0 0 0 7 0 0 0 0 0 0]
 [0 0 0 0 8 0 0 0 0 0]
 [0 0 0 0 0 8 0 0 0 0]
 [0 0 0 0 0 0 8 0 0 0]
 [0 0 0 0 0 0 0 8 0 0]
 [0 0 0 0 0 0 0 0 8 0]
 [0 0 0 0 0 0 0 0 0 7]]
